# Convolutional neural network

## Load X,y data from NPZ

Using the function added to *mylib.py* file, it's now easy to grab data and X/y vectors ready to be used for model training and tuning

In [1]:
# Run content of mylib.py file
%run mylib.py

# Load data from NPZ file
#data=loadNpz()
(data, X, y)=loadXy(verbose=False)

Before going further, let's talk a bit about *High Level Features*.

Wondering how those high level features are built, I've made some research on the Internet to find out how I could fit my own *Inception v3* high level features using the *small* dataset I have here.

As I will build my own Convolutional neural network, I will at the same time grab hish level features of 2048 bytes to be reused with one of the models build in the previous Notebooks.

Of course, the result will be extremly poor, I can not race against *Inception V3* fitting process on millions of pictures.

Anyway, this is fun to try, so let's do it.

    Note: High level feature extractions have been taken from https://www.tutorialspoint.com/how-can-keras-be-used-to-extract-features-from-only-one-layer-of-the-model-using-python


## Build a Conv Network and fit it


### Prepare the Dataset


In [2]:
import numpy as np


IMAGE_SIZE=data['IMAGE_SIZE']

IMAGE_COLOR_SPACE_DIM=data['IMAGE_COLOR_SPACE_DIM']

NB_LABELS=data['IMAGE_NB_LABELS']

FEATURE_SIZE=data['FEATURE_SIZE']

DATASET_NAME=data['DATASET_NAME']


KERAS_INPUT_SHAPE=(IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM)


X_tr=np.float32(data['trainX']['data'])
y_tr=data['trainX']['labels']

X_te=np.float32(data['test']['data'])
y_te=data['test']['labels']


### Import the needed libraries and define model

The model used here is taken from different exemples in the course.

The most important this to notice is the name of the 2048 Dense layer: *high-level-features*

This is from that layer that I will extract my 2048 high level features, to be used aginst a previous model.

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
from tensorflow.keras import activations

# Convolutional Network
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=2,
                              activation='relu', input_shape=KERAS_INPUT_SHAPE))
model.add(keras.layers.MaxPool2D(pool_size=3))
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=2048, activation='relu', name='high-level-features'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=512, activation='relu'))
model.add(keras.layers.Dense(units=6, activation='softmax'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
__________

### Compile and fit the model

optimzer, loss and EarlyStopping function are standard ones already used in previous Notebook.

In [4]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adamax(), loss='sparse_categorical_crossentropy', metrics=['acc'])

# End training when accuracy stops improving (optional)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

# Train model

model.fit(X_tr, y_tr, epochs=100, validation_split=0.2, callbacks=[early_stopping], verbose=1, workers=-1)

Train on 336 samples, validate on 84 samples
Epoch 1/100
336/336 [==============================] - 8s 24ms/step - loss: 3.5299 - acc: 0.2679 - val_loss: 2.0457 - val_acc: 0.0000e+00
Epoch 2/100
336/336 [==============================] - 4s 13ms/step - loss: 1.6667 - acc: 0.3185 - val_loss: 2.1381 - val_acc: 0.0000e+00
Epoch 3/100
336/336 [==============================] - 4s 13ms/step - loss: 1.5249 - acc: 0.4345 - val_loss: 1.9208 - val_acc: 0.1071
Epoch 4/100
336/336 [==============================] - 4s 13ms/step - loss: 1.3638 - acc: 0.5119 - val_loss: 1.8421 - val_acc: 0.1071
Epoch 5/100
336/336 [==============================] - 4s 13ms/step - loss: 1.1728 - acc: 0.5655 - val_loss: 1.8404 - val_acc: 0.1905
Epoch 6/100
336/336 [==============================] - 4s 13ms/step - loss: 0.9486 - acc: 0.6696 - val_loss: 1.5545 - val_acc: 0.3452
Epoch 7/100
336/336 [==============================] - 4s 13ms/step - loss: 0.6937 - acc: 0.7946 - val_loss: 1.7050 - val_acc: 0.3452
Epoch 8/1

### Evaluate the model

As expected, the score is very bad compared to the *Inception v3* high level features.

In [5]:
scores = model.evaluate(X_te, y_te, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 52.94%


### Save the model

It's now time to save the model in order to use it in the last Notebook of this project.


In [6]:
saveModel(model, 'cnn')

Saving model cnn to model-cnn.sav using 'keras.models.save_model' library


## So, what about high level features ?

As explained at the begining of this Notebook, I've decided to build my own high level features while fitting my Conv Network.

To do so, I will use the *keras.Model()* method to retrieve layers of my Conv Network, starting from the first one to the one named *high-level-features*

    Note: For more details on this, follow the weblink referenced at the begining of this Notebook.

In [7]:

feature_extractor = keras.Model(
   inputs=model.inputs,
   outputs=model.get_layer(name="high-level-features").output,
)


Summarizing the model now shows a Conv Net with the last layer being a Dense layer of 2048 bytes.

In [8]:
feature_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    (None, 299, 299, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16928)             0         
_________________________________________________________________
high-level-features (Dense)  (None, 2048)              34670592  
Total para

Now, to get a *high level feature* array of my dataset, I will simply use the *predict()* method of *keras.Model()* class on my test dataset.

I will get a matrix made of 51 entries (number of pictures in the test dataset) and 2048 bytes for each entry, which is the corresponding *high level feature*

In [9]:
features = feature_extractor.predict(X_te)
features.shape

(51, 2048)

To test my own features, let's load one of our previous model (the logistic one for exemple) and evaluate it.

In [10]:
model_to_test='knn'

accuracy=loadModel(model_to_test).score(features, y_te)

Loading model from  model-knn.sav
Model loaded using pickle()


In [11]:
accuracy=accuracy*100
print("  {} - Accuracy on test dataset using my own features: {:.1f}%\n".format(model_to_test, accuracy))

  knn - Accuracy on test dataset using my own features: 15.7%



Less than 16% accuracy, this is very bad :-)

Remember, this model performs 92% score using the *high level features* made from *Inception v3*

In [12]:
accuracy=loadModel(model_to_test).score(X['test'], y['test'])
print("Accuracy on test dataset using Inception v3 features: {:.1f}%".format(accuracy*100))

Loading model from  model-knn.sav
Model loaded using pickle()
Accuracy on test dataset using Inception v3 features: 92.2%


## Conclusion

Using models trained on huge dataset is the key to get the best accuracy for machine learning processes.

This has been demonstrated by building my own Convolution Network and compare result against all other models trained from the *Inception V3* high level features.

It's now time to go to our last Notebook to visualize the results taken from the variouis models trained in this project.